In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/modelllm/llm_model.h5
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/ai-generated-vs-human-generated/valid.csv
/kaggle/input/ai-generated-vs-human-generated/train.csv
/kaggle/input/ai-generated-vs-human-generated/merge.csv


In [2]:
import os
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import string
import tensorflow_text as tf_text

In [3]:
#train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
extra_train = pd.read_csv("/kaggle/input/ai-generated-vs-human-generated/merge.csv")
#extra_train_new = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv")
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
#train_prompts = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')

In [4]:
#train_final = pd.concat([extra_train_f, train.drop(columns=['prompt_id', 'id'], axis=0)], axis=0).reset_index(drop=True).drop_duplicates()
train_final = extra_train
train_final

,Unnamed: 0,prompt_name,text,generated,source,file
0,0,Phones and driving,Phones\n\nModern humans today are always on th...,0,persuade_corpus,daigt-v2-train-dataset/train_v2_drcat_02.csv
1,1,Phones and driving,This essay will explain if drivers should or s...,0,persuade_corpus,daigt-v2-train-dataset/train_v2_drcat_02.csv
2,2,Phones and driving,Driving while the use of cellular devices\n\nT...,0,persuade_corpus,daigt-v2-train-dataset/train_v2_drcat_02.csv
3,3,Phones and driving,Phones & Driving\n\nDrivers should not be able...,0,persuade_corpus,daigt-v2-train-dataset/train_v2_drcat_02.csv
4,4,Phones and driving,Cell Phone Operation While Driving\n\nThe abil...,0,persuade_corpus,daigt-v2-train-dataset/train_v2_drcat_02.csv
...,...,...,...,...,...,...
190339,190339,Does the electoral college work?,The electoral college iyitem ii a central aip...,1,NaN,llm-mistral-7b-instruct-texts/Mistral7B_CME_v7...
190340,190340,Does the electoral college work?,The electoral college iyitem ii a method uied...,1,NaN,llm-mistral-7b-instruct-texts/Mistral7B_CME_v7...
190341,190341,Does the electoral college work?,The elecioral college ii a iyiiem uied in ihe...,1,NaN,llm-mistral-7b-instruct-texts/Mistral7B_CME_v7...
190342,190342,Does the electoral college work?,The electoral college is a group of people ap...,1,NaN,llm-mistral-7b-instruct-texts/Mistral7B_CME_v7...


In [5]:
train_df, val_df = train_test_split(train_final, test_size=0.3, random_state=222)

batch_size = 32

raw_train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df['text'].values, train_df['generated'].values)
).batch(batch_size)

raw_val_ds = tf.data.Dataset.from_tensor_slices(
    (val_df['text'].values, val_df['generated'].values)
).batch(batch_size)

In [6]:
max_features = 75000
embedding_dim = 64
sequence_length = 512

def tf_lower_and_split_punct(text):
    text = tf_text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
    text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
    text = tf.strings.strip(text)
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text


# Text vectorization layer
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_features,
    ngrams = (3,5),
    output_mode="int",
    output_sequence_length=sequence_length,
    pad_to_max_tokens=True
)

text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [7]:
from tensorflow.keras.layers import TextVectorization, Embedding, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras import Model, Input

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [8]:
inputs = Input(shape=(sequence_length,), dtype="int64")
x = Embedding(max_features, embedding_dim)(inputs)
x = Bidirectional(LSTM(32, return_sequences=True))(x)
transformer_block = TransformerBlock(embedding_dim, 2, 32)
x = transformer_block(x)
x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid", name="predictions")(x)

model = Model(inputs=inputs, outputs=predictions)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 embedding (Embedding)       (None, 512, 64)           4800000   
                                                                 
 bidirectional (Bidirection  (None, 512, 64)           24832     
 al)                                                             
                                                                 
 transformer_block (Transfo  (None, 512, 64)           37664     
 rmerBlock)                                                      
                                                                 
 conv1d (Conv1D)             (None, 169, 128)          57472     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0     

In [9]:
loaded_model = tf.keras.models.load_model("/kaggle/input/modelllm/llm_model.h5", custom_objects={'TransformerBlock': TransformerBlock})

In [10]:
test_text = test['text'].values
vectorized_test_text = vectorize_layer(test_text)
predictions = model.predict(vectorized_test_text)

test['generated'] = predictions
print(test[['text', 'generated']])

1/1 [==============================] - 6s 6s/step
           text  generated
0  Aaa bbb ccc.   0.517842
1  Bbb ccc ddd.   0.517842
2  CCC ddd eee.   0.517842


In [11]:
test[['id', 'generated']].set_index('id').to_csv("submission.csv")